In [4]:
import tensorflow as tf
import tensorflow.keras as ks
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn import preprocessing

In [5]:
MEMORY_LIMIT = 1024
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=MEMORY_LIMIT)])
    except RuntimeError as e:
        print(e)

In [6]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
def load_data():
    
    trainDf = pd.read_csv('Exam02_data/Problem2/p2_train.csv')
    testDf = pd.read_csv('Exam02_data/Problem2/p2_test.csv')
    # dataframe to numpy arrays
    train = trainDf.to_numpy().astype("float32")
    test = testDf.to_numpy().astype("float32")
   
    # split features and labels
    X_train,y_train = train[:,:-1],train[:,-1]
    X_test, y_test = test[:,:-1],test[:,-1]
    
    # Normalizing data ( Mix-Max normalizing)
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train_minmax = min_max_scaler.fit_transform(X_train)
    X_test_minmax = min_max_scaler.transform(X_test)
    #print(X_train[0])
    #print(X_test[0])
    return X_train_minmax, y_train, X_test_minmax, y_test
x, _, _,_=load_data()


In [8]:
def build_model(input_space):
    ''' this function return a compiled model
    '''
    # add layers
    x = ks.layers.Input(shape=(input_space))
    hl = ks.layers.Dense(128,activation='relu')(x)
    hl = ks.layers.Dense(256,activation='relu')(hl)
    #hl = ks.layers.Dense(128,activation='relu')(hl)
    #hl = ks.layers.Dense(408,activation='relu')(hl)
    output = ks.layers.Dense(1, activation='sigmoid')(hl)
    
    model = ks.Model(inputs=x, outputs=output)
    model.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'] )
    return model

In [9]:
X_train,y_train,X_test, y_test = load_data()
model = build_model(X_train.shape[1])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1664      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 34,945
Trainable params: 34,945
Non-trainable params: 0
_________________________________________________________________


In [10]:
# 20% validation data
vlen = int(len(X_train)*0.2) 
x_val = X_train[-vlen:]
y_val = y_train[-vlen:]

history = model.fit(
    X_train,
    y_train,
    batch_size = 120,
    epochs = 100,
    validation_data=(x_val, y_val),
    verbose = 2
    )

Train on 349997 samples, validate on 69999 samples
Epoch 1/100


InternalError:  Blas GEMM launch failed : a.shape=(120, 12), b.shape=(12, 128), m=120, n=128, k=12
	 [[node model/dense/MatMul (defined at C:\Users\Rit1115\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_922]

Function call stack:
distributed_function


In [ ]:
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Epochs vs Accuracy")
plt.grid()


In [ ]:
results = model.evaluate(X_test,y_test)
print("\nTest accuracy: %.2f%%" %(results[1]*100))

In [ ]:
# Save the model
#model.save('my_model2_3hidden_128_256_128') 

In [ ]:
# Load the model 
new_model = tf.keras.models.load_model('my_model2_3hidden_128_256_128')

In [ ]:
new_model.summary()

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(X_test,  y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))